In [1]:
#### https://medium.com/better-programming/twitter-scrapers-are-all-broken-what-should-we-do-62a7349bfca6

from time import sleep
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import hashlib



In [2]:
chrome_driver = r"C:\Users\g\Documents\chrome_driver\chromedriver.exe"
driver = webdriver.Chrome(executable_path=chrome_driver)

# create master df to append to
#master_df = pd.DataFrame()



In [3]:
def sleep_for(opt1, opt2):
    time_for = random.uniform(opt1, opt2)
    time_for_int = int(round(time_for))
    sleep(abs(time_for_int - time_for))
    for i in range(time_for_int, 0, -1):
        sleep(1)

In [ ]:
class TwitterScraper:
    def __init__(self):
        self.tweetsdf = pd.DataFrame(columns=['handle','datetime','text'])
        self.logged_set = set()
        
    def addTweet(self,handle,datetime,text):
        tweethash = hashlib.sha1( datetime.encode('ANSI') + text.encode('ANSI') + handle.encode('ANSI') ).hexdigest()
        if tweethash not in self.logged_set:
            self.tweetsdf = self.tweetsdf.append( pd.DataFrame(data=[[handle,datetime,text]], columns=['handle','datetime','text']) )
            return(1)
        else:
            self.logged_set.add(tweethash)
            return(0)
            
    
    def get_tweets(self, url, scrolls, max_strikes=2):
        driver.get(url)
        sleep_for(4, 5)

        tweet_xpath = '//div/div/article/div/div'

        already_logged = set()
        strikes = 0

        for i in range(0, scrolls):

            # get a list of tweets
            post_list = driver.find_elements_by_xpath(tweet_xpath)

            # prase tweet metadata
            for post in post_list:
                tweets_found = 0
                try:
                    time_xpath = './/div/div[2]/div[2]/div[1]/div/div/div[1]/a/time'
                    text_xpath = './/div/div[2]/div[2]/div[2]/div[1]/div'
                    handle_xpath = './/div/div[2]/div[2]/div[1]/div/div/div[1]/div[1]/a/div/div[2]/div/span'
                    handle = post.find_element_by_xpath(handle_xpath).text
                    datetime = post.find_element_by_xpath(time_xpath).get_attribute('datetime')
                    text = post.find_element_by_xpath(text_xpath).text

                    tweets_found += self.addTweet(handle,datetime,text)
                    
                except:
                    # likely an ad
                    pass

            if tweets_found == 0:
                if strikes > max_strikes:
                    break
                strikes += 1
            else:
                strikes = 0

            # scroll down
            driver.find_element_by_xpath('//body').send_keys(Keys.END)
            sleep_for(2, 4)

    
    def get_search(self, query, scrolls=5):
        query = query.replace(' ', '%20')
        url = 'https://twitter.com/search?q={}&src=typed_query'.format(query)
        self.get_tweets(url, scrolls, max_strikes=2)
        

    def get_user(self, handle, scrolls):
        url = 'https://twitter.com/{}'.format(handle)
        self.get_tweets(url, scrolls, max_strikes=2)
        
    def get_df(self):
        return(self.tweetsdf)
    
test = TwitterScraper()
test.get_user('BillGates',scrolls=5)
test.get_df()